# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [25]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/luka/jupyterenv/lib/python3.12/site-packages/IPython/core/magics/pylab.py:166: UserWarning: pylab import has clobbered these variables: ['clf', 'permutation']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [26]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Belly', 'StandInit', 'Back', 'Crouch', 'Left', 'Stand', 'Frog', 'Knee', 'Sit', 'HeadBack', '.ipynb_checkpoints', 'Right']


In [93]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    if classes[i] == '.ipynb_checkpoints':
        pass
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    raw_data = pickle.load(open(filename, 'rb'))
    filtered = [row for row in raw_data if all(isinstance(v, float) for v in row)]
    

    data.extend(filtered) 
    target.extend([i] * len(filtered))  
        
    
    return data, target

In [102]:
# load all the data
all_data = []
all_target = []
for i in range(len(classes)):
    if classes[i] == '.ipynb_checkpoints':
        continue
    datas, targets = load_pose_data(i)
    all_data.extend(datas)
    all_target.extend(targets)
 
# YOUR CODE HERE
X = np.array(all_data)  
y = np.array(all_target)

print(X.shape)
print(X[0])
print ('total number of data', len(all_data))

print(f"Final data shape: {X.shape}, Target shape: {y.shape}")

(222, 10)
[ 0.06600404  0.0061779   0.0798099   2.00796413  0.06600404  0.08441186
  0.09506607  1.89760017 -0.01857978  1.49046504]
total number of data 222
Final data shape: (222, 10), Target shape: (222,)


In [103]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [104]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [105]:
# YOUR CODE HERE
clf.fit(X[training_data],y[training_data])




SVC(C=100.0, gamma=0.001)

### predicting

In [106]:
clf.predict(X[-1].reshape(1,-1)), y[-1].reshape(1,-1)

(array([3]), array([[11]]))

In [107]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [101]:
expected = []
predicted = []
# YOUR CODE HERE
expected=X[training_data]
predicted=clf.predict(y[training_data])

evaluate(expected, predicted)

ValueError: Expected 2D array, got 1D array instead:
array=[ 2.  2.  8. 11.  0.  1.  7.  6.  2.  1.  5.  5.  4.  8.  3.  8.  3.  3.
  1.  9.  9.  1.  8.  2.  8.  1.  1.  2.  1. 11.  8. 11.  2.  2.  1.  5.
  1.  3.  9.  4.  8.  8.  1.  4.  3.  1.  1.  0.  6.  8.  1.  4.  3. 11.
  1.  8.  8. 11.  1.  2.  3.  1.  0.  3.  1.  2.  2.  3.  3.  7.  1.  1.
  4.  4.  0.  3.  1.  5.  9.  7.  0.  1.  3.  3.  8.  4.  0.  1.  8.  2.
  1.  0.  8.  7.  2.  4. 11. 11.  9.  3.  8.  8.  2.  5.  6.  1.  1.  0.
  4.  3.  8.  3.  3.  8.  0.  4.  8.  1.  9.  9.  2.  6.  8.  8. 11.  6.
  5.  5.  3.  3.  8.  2.  3.  0.  5.  4.  4.  1. 11.  0.  3.  1.  1.  1.
  3.  1.  7.  1.  7. 11.  0.  0.  4.  3.  5.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [90]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [91]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[:-1]), all_target[:-1]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
         7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]),
 [0,
  0,
